In [1]:
import sys
sys.path.append("./../../src")
import pandas as pd
import numpy as np

import torch
from torch import nn, optim
import pickle
from icd9_tree.icd9 import ICD9

from vae_with_dataloader import VAE, VAETrainerWithDataLoader
# from icd_analysis_helper import ICDAnalysisHelper
from dataloader import MixehrICDImputationDataset
from visualizer_helper import Visualizer

In [2]:
icd9_cols_path = "/Users/andrew/Documents/meng/spring/PATIENT_ICD_ICD9_CODES.csv"
subject_ids_col_path = "/Users/andrew/Documents/meng/spring/PATIENT_ICD_SUBJECT_IDS.csv"
local_icd_path_diag = "/Users/andrew/Documents/meng/mimic-iii-clinical-database-1.4/D_ICD_DIAGNOSES.csv.gz"
local_icd_path_proc = "/Users/andrew/Documents/meng/mimic-iii-clinical-database-1.4/D_ICD_PROCEDURES.csv.gz"
diag_path = "/Users/andrew/Documents/meng/mimic-iii-clinical-database-1.4/DIAGNOSES_ICD.csv.gz"

icd9_col = pd.read_csv(icd9_cols_path)
subject_ids_col = pd.read_csv(subject_ids_col_path)
icds_diag_df = pd.read_csv(local_icd_path_diag)
icds_proc_df = pd.read_csv(local_icd_path_proc)

diag = pd.read_csv(diag_path)
'0030' in diag.ICD9_CODE.tolist()

True

In [3]:
icd9_tree = ICD9('./../../src/icd9_tree/codes.json')
toplevelnodes = [node.code for node in icd9_tree.children]

In [4]:
toplevelnodes = [node.code for node in icd9_tree.children]

icd9_tree.find("010.85")
for code in icd9_col.ICD9_CODE:
    if icd9_tree.find(code) == None:
        obj_found = False
        while len(code) > 0 and not obj_found:
            print("Couldn't find {}".format(code))
            code = code[:-1]
            if icd9_tree.find(code) != None:
                print("    Found for {}".format(code))
                obj_found = True
            
    

Couldn't find 01085
    Found for 0108
Couldn't find 01123
    Found for 0112
Couldn't find 01136
    Found for 0113
Couldn't find 01164
    Found for 0116
Couldn't find 01186
    Found for 0118
Couldn't find 01190
    Found for 0119
Couldn't find 01193
    Found for 0119
Couldn't find 01194
    Found for 0119
Couldn't find 01205
    Found for 0120
Couldn't find 01215
    Found for 0121
Couldn't find 01300
    Found for 0130
Couldn't find 01304
    Found for 0130
Couldn't find 01325
    Found for 0132
Couldn't find 01330
    Found for 0133
Couldn't find 01354
    Found for 0135
Couldn't find 01402
    Found for 0140
Couldn't find 01405
    Found for 0140
Couldn't find 01485
    Found for 0148
Couldn't find 01504
    Found for 0150
Couldn't find 01505
    Found for 0150
Couldn't find 01803
    Found for 0180
Couldn't find 01805
    Found for 0180
Couldn't find 01880
    Found for 0188
Couldn't find 01890
    Found for 0189
Couldn't find 01894
    Found for 0189
Couldn't find 01895
    F

Couldn't find 27400
    Found for 2740
Couldn't find 27401
    Found for 2740
Couldn't find 27402
    Found for 2740
Couldn't find 27403
    Found for 2740
Couldn't find 27501
    Found for 2750
Couldn't find 27502
    Found for 2750
Couldn't find 27503
    Found for 2750
Couldn't find 27509
    Found for 2750
Couldn't find 27661
    Found for 2766
Couldn't find 27669
    Found for 2766
Couldn't find 27788
    Found for 2778
Couldn't find 27803
    Found for 2780
Couldn't find 27941
    Found for 2794
Couldn't find 27949
    Found for 2794
Couldn't find 2800
    Found for 280
Couldn't find 2808
    Found for 280
Couldn't find 2809
    Found for 280
Couldn't find 2810
    Found for 281
Couldn't find 2811
    Found for 281
Couldn't find 2812
    Found for 281
Couldn't find 2813
    Found for 281
Couldn't find 2818
    Found for 281
Couldn't find 2819
    Found for 281
Couldn't find 2820
    Found for 282
Couldn't find 2821
    Found for 282
Couldn't find 2822
    Found for 282
Couldn't f

    Found for 3056
Couldn't find 30570
    Found for 3057
Couldn't find 30571
    Found for 3057
Couldn't find 30573
    Found for 3057
Couldn't find 30580
    Found for 3058
Couldn't find 30583
    Found for 3058
Couldn't find 30590
    Found for 3059
Couldn't find 30591
    Found for 3059
Couldn't find 30592
    Found for 3059
Couldn't find 30593
    Found for 3059
Couldn't find 34200
    Found for 3420
Couldn't find 34201
    Found for 3420
Couldn't find 34202
    Found for 3420
Couldn't find 34210
    Found for 3421
Couldn't find 34211
    Found for 3421
Couldn't find 34212
    Found for 3421
Couldn't find 34280
    Found for 3428
Couldn't find 34281
    Found for 3428
Couldn't find 34282
    Found for 3428
Couldn't find 34290
    Found for 3429
Couldn't find 34291
    Found for 3429
Couldn't find 34292
    Found for 3429
Couldn't find 34500
    Found for 3450
Couldn't find 34501
    Found for 3450
Couldn't find 34510
    Found for 3451
Couldn't find 34511
    Found for 3451
Couldn

Couldn't find 55000
    Found for 5500
Couldn't find 55001
    Found for 5500
Couldn't find 55010
    Found for 5501
Couldn't find 55011
    Found for 5501
Couldn't find 55012
    Found for 5501
Couldn't find 55090
    Found for 5509
Couldn't find 55091
    Found for 5509
Couldn't find 55092
    Found for 5509
Couldn't find 55093
    Found for 5509
Couldn't find 55841
Couldn't find 5584
    Found for 558
Couldn't find 56032
    Found for 5603
Couldn't find 56481
Couldn't find 5648
    Found for 564
Couldn't find 56489
Couldn't find 5648
    Found for 564
Couldn't find 56962
    Found for 5696
Couldn't find 56971
Couldn't find 5697
    Found for 569
Couldn't find 56979
Couldn't find 5697
    Found for 569
Couldn't find 56987
    Found for 5698
Couldn't find 5735
    Found for 573
Couldn't find 57400
    Found for 5740
Couldn't find 57401
    Found for 5740
Couldn't find 57410
    Found for 5741
Couldn't find 57411
    Found for 5741
Couldn't find 57420
    Found for 5742
Couldn't find 5

Couldn't find 71101
    Found for 7110
Couldn't find 71102
    Found for 7110
Couldn't find 71103
    Found for 7110
Couldn't find 71104
    Found for 7110
Couldn't find 71105
    Found for 7110
Couldn't find 71106
    Found for 7110
Couldn't find 71107
    Found for 7110
Couldn't find 71108
    Found for 7110
Couldn't find 71109
    Found for 7110
Couldn't find 71120
    Found for 7112
Couldn't find 71155
    Found for 7115
Couldn't find 71180
    Found for 7118
Couldn't find 71191
    Found for 7119
Couldn't find 71195
    Found for 7119
Couldn't find 71198
    Found for 7119
Couldn't find 71213
    Found for 7121
Couldn't find 71216
    Found for 7121
Couldn't find 71222
    Found for 7122
Couldn't find 71226
    Found for 7122
Couldn't find 71230
    Found for 7123
Couldn't find 71233
    Found for 7123
Couldn't find 71235
    Found for 7123
Couldn't find 71236
    Found for 7123
Couldn't find 71237
    Found for 7123
Couldn't find 71238
    Found for 7123
Couldn't find 71239
    F

Couldn't find 7456
    Found for 745
Couldn't find 74561
Couldn't find 7456
    Found for 745
Couldn't find 74569
Couldn't find 7456
    Found for 745
Couldn't find 7458
    Found for 745
Couldn't find 7459
    Found for 745
Couldn't find 74600
Couldn't find 7460
    Found for 746
Couldn't find 74601
Couldn't find 7460
    Found for 746
Couldn't find 74602
Couldn't find 7460
    Found for 746
Couldn't find 74609
Couldn't find 7460
    Found for 746
Couldn't find 7461
    Found for 746
Couldn't find 7462
    Found for 746
Couldn't find 7463
    Found for 746
Couldn't find 7464
    Found for 746
Couldn't find 7466
    Found for 746
Couldn't find 7467
    Found for 746
Couldn't find 74681
Couldn't find 7468
    Found for 746
Couldn't find 74682
Couldn't find 7468
    Found for 746
Couldn't find 74683
Couldn't find 7468
    Found for 746
Couldn't find 74684
Couldn't find 7468
    Found for 746
Couldn't find 74685
Couldn't find 7468
    Found for 746
Couldn't find 74686
Couldn't find 7468
 

Couldn't find 76401
    Found for 7640
Couldn't find 76402
    Found for 7640
Couldn't find 76403
    Found for 7640
Couldn't find 76404
    Found for 7640
Couldn't find 76405
    Found for 7640
Couldn't find 76406
    Found for 7640
Couldn't find 76407
    Found for 7640
Couldn't find 76408
    Found for 7640
Couldn't find 76409
    Found for 7640
Couldn't find 76416
    Found for 7641
Couldn't find 76418
    Found for 7641
Couldn't find 76491
    Found for 7649
Couldn't find 76492
    Found for 7649
Couldn't find 76493
    Found for 7649
Couldn't find 76494
    Found for 7649
Couldn't find 76495
    Found for 7649
Couldn't find 76496
    Found for 7649
Couldn't find 76497
    Found for 7649
Couldn't find 76498
    Found for 7649
Couldn't find 76499
    Found for 7649
Couldn't find 76500
    Found for 7650
Couldn't find 76501
    Found for 7650
Couldn't find 76502
    Found for 7650
Couldn't find 76503
    Found for 7650
Couldn't find 76504
    Found for 7650
Couldn't find 76505
    F

Couldn't find 81000
    Found for 8100
Couldn't find 81001
    Found for 8100
Couldn't find 81002
    Found for 8100
Couldn't find 81003
    Found for 8100
Couldn't find 81010
    Found for 8101
Couldn't find 81012
    Found for 8101
Couldn't find 81013
    Found for 8101
Couldn't find 81100
    Found for 8110
Couldn't find 81101
    Found for 8110
Couldn't find 81102
    Found for 8110
Couldn't find 81103
    Found for 8110
Couldn't find 81109
    Found for 8110
Couldn't find 81110
    Found for 8111
Couldn't find 81400
    Found for 8140
Couldn't find 81401
    Found for 8140
Couldn't find 81402
    Found for 8140
Couldn't find 81403
    Found for 8140
Couldn't find 81404
    Found for 8140
Couldn't find 81405
    Found for 8140
Couldn't find 81406
    Found for 8140
Couldn't find 81407
    Found for 8140
Couldn't find 81408
    Found for 8140
Couldn't find 81409
    Found for 8140
Couldn't find 81411
    Found for 8141
Couldn't find 81412
    Found for 8141
Couldn't find 81418
    F

Couldn't find 88000
    Found for 8800
Couldn't find 88001
    Found for 8800
Couldn't find 88002
    Found for 8800
Couldn't find 88003
    Found for 8800
Couldn't find 88012
    Found for 8801
Couldn't find 88013
    Found for 8801
Couldn't find 88019
    Found for 8801
Couldn't find 88020
    Found for 8802
Couldn't find 88023
    Found for 8802
Couldn't find 88029
    Found for 8802
Couldn't find 88100
    Found for 8810
Couldn't find 88101
    Found for 8810
Couldn't find 88102
    Found for 8810
Couldn't find 88110
    Found for 8811
Couldn't find 88111
    Found for 8811
Couldn't find 88112
    Found for 8811
Couldn't find 88120
    Found for 8812
Couldn't find 88121
    Found for 8812
Couldn't find 88122
    Found for 8812
Couldn't find 9100
Couldn't find 910
Couldn't find 91
Couldn't find 9
Couldn't find 9102
Couldn't find 910
Couldn't find 91
Couldn't find 9
Couldn't find 9104
Couldn't find 910
Couldn't find 91
Couldn't find 9
Couldn't find 9108
Couldn't find 910
Couldn't fin

Couldn't find E8171
    Found for E817
Couldn't find E8178
    Found for E817
Couldn't find E8180
    Found for E818
Couldn't find E8181
    Found for E818
Couldn't find E8182
    Found for E818
Couldn't find E8187
    Found for E818
Couldn't find E8188
    Found for E818
Couldn't find E8189
    Found for E818
Couldn't find E8190
    Found for E819
Couldn't find E8191
    Found for E819
Couldn't find E8192
    Found for E819
Couldn't find E8193
    Found for E819
Couldn't find E8196
    Found for E819
Couldn't find E8197
    Found for E819
Couldn't find E8199
    Found for E819
Couldn't find E8200
    Found for E820
Couldn't find E8210
    Found for E821
Couldn't find E8211
    Found for E821
Couldn't find E8212
    Found for E821
Couldn't find E8216
    Found for E821
Couldn't find E8217
    Found for E821
Couldn't find E8219
    Found for E821
Couldn't find E8226
    Found for E822
Couldn't find E8227
    Found for E822
Couldn't find E8228
    Found for E822
Couldn't find E8230
    F

In [5]:
icd9_tree.find("010.8").description

AttributeError: 'NoneType' object has no attribute 'description'

In [ ]:
k = 10

a = icd9_tree.find(icd9_col['ICD9_CODE'].iloc[1])
print(a)